In [ ]:
import os
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import pickle

# Function to encode mutations
def encode_mutation(mutation):
    if '->' in mutation:
        # insertions are marked as 1 for the first bit, and then a concatenation of its bases
        inserted_sequence = mutation.split('->')[1]
        mutation_encoded = [1]  # Set the first bit to 1 for insertion
        for base in inserted_sequence:
            if base == 'A':
                mutation_encoded.extend([0, 0])
            elif base == 'T':
                mutation_encoded.extend([0, 1])
            elif base == 'G':
                mutation_encoded.extend([1, 0])
            elif base == 'C':
                mutation_encoded.extend([1, 1])
    elif '>-' in mutation:
        # deletions are marked as 0 for the first bit, and then a concatenation of its bases
        deleted_sequence = mutation.split('>-')[0].split(':')[2]
        mutation_encoded = [0]  # Set the first bit to 0 for deletion
        for base in deleted_sequence:
            if base == 'A':
                mutation_encoded.extend([0, 0])
            elif base == 'T':
                mutation_encoded.extend([0, 1])
            elif base == 'G':
                mutation_encoded.extend([1, 0])
            elif base == 'C':
                mutation_encoded.extend([1, 1])
    elif '>' in mutation:
        #  subtitutions are encoded as -1 for start, 1 for end, 0 else
        mutation_info = mutation.split(':')[2]
        start_nucleotide, end_nucleotide = mutation_info.split('>')[0], mutation_info.split('>')[1]
        mutation_encoded = [-1 if nuc == start_nucleotide else (1 if nuc == end_nucleotide else 0) for nuc in ['A', 'T', 'G', 'C']]
    else:
        # For other types of mutations (e.g., duplications, inversions)
        mutation_encoded = [0] * (len(mutation) * 2)  # Set all bits to 0

    return mutation_encoded

# Function to encode consequences
def encode_consequence(consequence):
    if consequence in consequence_mapping:
        return consequence_mapping[consequence]
    else:
        # Assign a new numerical representation for 'other' consequence types
        return len(consequence_mapping)

# Function to pad mutation sequences
def pad_mutation_sequence(mutation_encoded):
    mutation_length = len(mutation_encoded)
    if mutation_length < max_mutation_length:
        padding_length = max_mutation_length - mutation_length
        mutation_encoded.extend([2] * padding_length)  # Padding with 2s because AA is encoded as 00, and transitions use -1
    return mutation_encoded

# Function to process a single TSV file
def process_tsv(file_path):
    # Load the DataFrame from the TSV file
    df = pd.read_csv(file_path, delimiter="\t")

    # Apply encoding and preprocessing steps
    df['Mutation Encoded'] = df['Mutation (GRCh38)'].apply(encode_mutation)
    df['Consequence'] = df['Consequence'].apply(encode_consequence)
    df['Mutation Encoded'] = df['Mutation Encoded'].apply(pad_mutation_sequence)
    # Extract chromosome number from mutation and hot encode
    df['Chromosome'] = df['Mutation (GRCh38)'].str.split(':').str[0]
    df['Chromosome'] = df['Chromosome'].astype(int)
    df['Chromosome'] = df['Chromosome'].apply(lambda x: format(x, '06b'))  # Convert to 6-bit binary

    # Extract base pair position from mutation and normalize
    df['Base Pair Position'] = df['Mutation (GRCh38)'].str.split(':').str[1].astype(int)
    scaler = MinMaxScaler()
    df['Normalized Base Pair Position'] = scaler.fit_transform(df[['Base Pair Position']])

    # Drop original mutation and base pair position columns
    df.drop(['Mutation (GRCh38)', 'Protein Position', 'Base Pair Position'], axis=1, inplace=True)

    return df

# Define the folder containing the TSV files
folder_path = "/content/drive/MyDrive/NLP_DNA_TrainingData/"

# Define the known consequence types and their numerical representations (these are the targets)
consequence_mapping = {
    'missense_variant': 0,
    'stop_gained': 1,
    'frameshift_variant': 2,
    'splice_acceptor_variant': 3,
    'splice_donor_variant': 4,
    'synonymous_variant': 5,
    'inframe_deletion': 6,
    'stop_gained': 7
}

# Define the maximum mutation sequence length (determined based on your data)
max_mutation_length = 20  # Adjust this value to match the length of the maximum insertion/deletion sequence
# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".tsv"):
        file_path = os.path.join(folder_path, file_name)
        # Process the TSV file
        processed_df = process_tsv(file_path)

        # Save the processed DataFrame as a Pickle file for training
        output_file_path = os.path.join("/content/drive/MyDrive/NLP_DNA_PKLs", f"{file_name[:-4]}.pkl")
        with open(output_file_path, 'wb') as f:
            pickle.dump(processed_df, f)